In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5507567214687947, 0.7062927543459533)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns

In [6]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.1, cols, 816)
single_stump_model['features']

['age_at_current_charge21.0',
 'age_at_current_charge29.0',
 'age_at_current_charge30.0',
 'age_at_current_charge33.0',
 'age_at_current_charge69.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_arrest10',
 'p_charges1',
 'p_charges2',
 'p_charges5',
 'p_violence2',
 'p_felony2',
 'p_misdemeanor3',
 'p_property3',
 'p_felprop_viol1',
 'p_drug3',
 'p_dui1',
 'p_pending_charge5',
 'p_probation3',
 'p_incarceration1',
 'three_year1']

In [7]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [8]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.1}, 0.6890722194007219, 0.0400723760160675)

### RiskSLIM

In [9]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns

In [25]:
single_stump_model = stumps.stump_model(FL_X, 
                                        FL_Y, 
                                        KY_X, 
                                        KY_Y, 
                                        0.05, 
                                        cols, 816)
selected_features = single_stump_model['features']
len(selected_features)

17

In [26]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [23]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'felony_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 12:27 AM | 223 rows in lookup table
11/12/19 @ 12:27 AM | ------------------------------------------------------------
11/12/19 @ 12:27 AM | runnning initialization procedure
11/12/19 @ 12:27 AM | ------------------------------------------------------------
11/12/19 @ 12:27 AM | CPA produced 2 cuts
11/12/19 @ 12:27 AM | running naive rounding on 4 solutions
11/12/19 @ 12:27 AM | best objective value: 0.6931
11/12/19 @ 12:27 AM | rounding produced 4 integer solutions
11/12/19 @ 12:27 AM | best objective value is 0.6931
11/12/19 @ 12:27 AM | running sequential rounding on 4 solutions
11/12/19 @ 12:27 AM | best objective value: 0.6931
11/12/19 @ 12:27 AM | sequential rounding produced 0 integer solutions
11/12/19 @ 12:27 AM | polishing 4 solutions
11/12/19 @ 12:27 AM | best objective value: 0.6931
11/12/19 @ 12:27 AM | polishing produced 4 integer solutions
11/12/19 @ 12:27 AM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 12:27 AM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4508.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 828000 68360        0.4400     6        0.4488        0.4400  2081375    1.95%           rho_7 U 828000 827995     38
 846358 69863        0.4415     1        0.4488        0.4401  2125362    1.94%           rho_4 U 846358 846356     32
 864597 72050        cutoff              0.4488        0.4401  2168680    1.93%           rho_3 D 864597 864596     47
 884762 73765        0.4422     2        0.4488        0.4402  2210707    1.91%           rho_0 D 884762 884760     42
Elapsed time = 153.14 sec. (166993.83 ticks, tree = 22.79 MB, solutions = 5)
 906735 74705        0.4449     4        0.4488        0.4403  2252283    1.89%           rho_8 U 906735 906734     37
 926114 76506        0.4410     2        0.4488        0.4403  2288880    1.88%           rho_5 U 926114 926112     38
 945560 78256        cutoff              0.4488        0.4403  2321764    1.88%           rho_0 U 945560 945559     49
 964970 79979        0.4446     1        0.4488        0.4403  2355532    1.88%           

11/12/19 @ 12:35 AM | best objective value: 0.6931
11/12/19 @ 12:35 AM | polishing produced 4 integer solutions
11/12/19 @ 12:35 AM | initialization produced 7 feasible solutions
11/12/19 @ 12:35 AM | best objective value: 0.4528
11/12/19 @ 12:35 AM | ------------------------------------------------------------
11/12/19 @ 12:35 AM | completed initialization procedure
11/12/19 @ 12:35 AM | ------------------------------------------------------------
11/12/19 @ 12:35 AM | 223 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.43637640617764994
CPXPARAM_MIP_Tolerances_UpperCutoff              0.45282673732478051


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 12:35 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4528.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

 887763 112579        0.4544     1        0.4587        0.4488  3055585    2.16%           rho_7 D 887763 887761     40
 908316 114863        0.4491     9        0.4587        0.4489  3117011    2.13%           rho_7 D 908316 908314     38
 928249 117746        cutoff              0.4587        0.4489  3170791    2.13%           rho_0 D 928249 928247     58
 948257 119216        0.4531     5        0.4587        0.4489  3224778    2.12%          rho_14 D 948257 948256     49
Elapsed time = 118.08 sec. (166579.64 ticks, tree = 32.11 MB, solutions = 5)
 966434 118494        0.4498     7        0.4587        0.4490  3279548    2.11%           rho_8 U 966434 966432     46
 986291 121352        cutoff              0.4587        0.4490  3342540    2.10%           rho_6 D 986291 986289     53
 1008982 125518        0.4490     8        0.4587        0.4490  3400290    2.10%           rho_7 U 1008982 1008971     47
*1021800+126553                            0.4587        0.4490             2.10

11/12/19 @ 01:19 AM | ------------------------------------------------------------
11/12/19 @ 01:19 AM | runnning initialization procedure
11/12/19 @ 01:19 AM | ------------------------------------------------------------
11/12/19 @ 01:19 AM | CPA produced 2 cuts
11/12/19 @ 01:19 AM | running naive rounding on 4 solutions
11/12/19 @ 01:19 AM | best objective value: 0.6931
11/12/19 @ 01:19 AM | rounding produced 4 integer solutions
11/12/19 @ 01:19 AM | best objective value is 0.6931
11/12/19 @ 01:19 AM | running sequential rounding on 4 solutions
11/12/19 @ 01:19 AM | best objective value: 0.6931
11/12/19 @ 01:19 AM | sequential rounding produced 0 integer solutions
11/12/19 @ 01:19 AM | polishing 4 solutions
11/12/19 @ 01:19 AM | best objective value: 0.6931
11/12/19 @ 01:19 AM | polishing produced 4 integer solutions
11/12/19 @ 01:19 AM | initialization produced 7 feasible solutions
11/12/19 @ 01:19 AM | best objective value: 0.4559
11/12/19 @ 01:19 AM | -----------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 01:19 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4559.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 753547 77939        cutoff              0.4536        0.4423  2188242    2.48%           rho_8 N 753547 589712     32
 772288 79443        0.4425     7        0.4536        0.4425  2236879    2.45%           rho_0 D 772288 772272     28
 790888 82170        0.4451     1        0.4536        0.4425  2283185    2.45%           rho_3 D 790888 790886     37
 809090 83248        0.4435     4        0.4536        0.4425  2332061    2.44%           rho_5 U 809090 809088     45
 828313 85323        0.4425     4        0.4536        0.4425  2391923    2.44%           rho_3 D 828313 828311     45
 847813 86942        0.4425     6        0.4536        0.4425  2447459    2.44%           rho_5 D 847813 754103     29
Elapsed time = 108.41 sec. (168105.83 ticks, tree = 25.74 MB, solutions = 5)
 866129 89458        cutoff              0.4536        0.4426  2496611    2.44%           rho_8 N 866129 866128     36
 884038 91304        0.4515     1        0.4536        0.4426  2547632    2.43%          r

 2997990 152159        0.4472     4        0.4536        0.4472  7694512    1.41%           rho_3 D 2997990 2997981     32
Elapsed time = 358.66 sec. (625701.51 ticks, tree = 46.70 MB, solutions = 7)
 3015228 151922        0.4473     7        0.4536        0.4473  7733838    1.39%           rho_5 U 3015228 3015227     33
 3032279 150950        0.4473     4        0.4536        0.4473  7776641    1.38%           rho_0 D 3032279 3032278     25
 3051548 150665        0.4474     1        0.4536        0.4474  7814896    1.36%           rho_8 U 3051548 3051521     37
 3068798 150447        0.4501     5        0.4536        0.4474  7850819    1.36%           rho_0 N 3068798 2968929     32
 3087644 150182        0.4480     1        0.4536        0.4475  7882419    1.35%           rho_0 D 3087644 3087643     34
 3105399 150098        cutoff              0.4536        0.4475  7921114    1.34%           rho_5 U 3105399 3105398     19
 3127361 152158        0.4475     1        0.4536        0.447

In [24]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.513644145212071, 0.5911421537246424)

In [28]:
#### save results
summary_felony2_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}

Stored 'summary_felony2_fl_inter_model' (dict)


In [28]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Felony",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)